In [6]:
from transformers import (TextDataset, DataCollatorForLanguageModeling,GPT2Tokenizer,
                          GPT2LMHeadModel,Trainer, TrainingArguments)
import pandas as pd
import matplotlib.pyplot as plt
import wandb
wandb.init(mode="disabled")
import warnings
warnings.filterwarnings('ignore')
import json
import os
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import streamlit as st
import torch
from torch.nn import DataParallel

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-21 14:34:25.416092: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-21 14:34:25.416138: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-21 14:34:25.417301: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-21 14:34:25.424410: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to 

In [7]:
#read the response_1.csv file and drop the null values and last column
df1 = pd.read_csv('../data/sample_q_and_a/response_1.csv')
print("Shape before dropping", df1.shape)
df1 = df1.dropna()
print("Shape after dropping", df1.shape)

#drop the last column
df1 = df1.drop(df1.columns[-1], axis=1)
print("Shape after dropping last column", df1.shape)

#change the column names in to Question	and Response
df1 = df1.rename(columns={'question':'Question', 'response_j':'Response'})
print(df1.head())
print("Shape of df1",df1.shape)


Shape before dropping (9846, 3)
Shape after dropping (9846, 3)
Shape after dropping last column (9846, 2)
                                            Question  \
0  I'm feeling really anxious lately and I don't ...   
1  I think my partner may be cheating on me. What...   
2  I'm feeling really overwhelmed with work and s...   
3  I'm having trouble sleeping and I'm constantly...   
4  I've been feeling really anxious lately, and I...   

                                            Response  
0  It's common to feel anxious at times, and ther...  
1  It's understandable to feel worried and suspic...  
2  It sounds like you're going through a difficul...  
3  It's important to talk to your doctor about an...  
4  It's common to feel anxious without knowing th...  
Shape of df1 (9846, 2)


In [8]:
#read the rules_facts.csv file and drop the null values and last column
df2 = pd.read_csv('../data/sample_q_and_a/rules_facts.csv')
print("Shape before dropping", df2.shape)
df2 = df2.dropna()
print("Shape after dropping", df2.shape)

#drop the last column
df2 = df2.drop(df2.columns[-2], axis=1)
print("Shape after dropping last column", df2.shape)

#change the column names in to Question	and Response
df2 = df2.rename(columns={'Context':'Question'})
print(df2.head())
print("Shape of df2",df2.shape)

Shape before dropping (661, 3)
Shape after dropping (657, 3)
Shape after dropping last column (657, 2)
  Question                                           Response
0       Hi    Hello there. Tell me how are you feeling today?
1       Hi              Hi there. What brings you here today?
2       Hi               Hi there. How are you feeling today?
3       Hi       Great to see you. How do you feel currently?
4       Hi  Hello there. Glad to see you're back. What's g...
Shape of df2 (657, 2)


In [9]:
#read the rules_facts.csv file and drop the null values and last column
df3 = pd.read_csv('../data/knowledge/all_types.csv')
print("Shape before dropping", df3.shape)
df3 = df3.dropna()
print("Shape after dropping", df3.shape)

#drop the last column
df3 = df3.drop(df3.columns[0], axis=1)
print("Shape after dropping last column", df3.shape)

#change the column names in to Question	and Response
df3 = df3.rename(columns={'selftext':'Response'})
df3 = df3.rename(columns={'subreddit':'Question'})
print(df3.head())
# Take only the random 100 rows for each unique 'Question'
df3_grouped = df3.groupby('Question').sample(100)

print("Shape after taking 100 rows for each unique 'Question'", df3_grouped.shape)

#move the Question column to the first column
df3_grouped = df3_grouped[['Question', 'Response']]
print(df3_grouped.head())
print("Shape of df3",df3_grouped.shape)


Shape before dropping (853075, 3)
Shape after dropping (817733, 3)
Shape after dropping last column (817733, 2)
                                            Response Question
0  A few months ago I was accepted into this full...     ADHD
1  Hey guys, I was curious if anyone else has the...     ADHD
2   \n\ni have 6 exams in the next 2 weeks one of...     ADHD
3  Is there anyone out there that is struggling w...     ADHD
4  Whenever I get hungry, I never eat because I d...     ADHD
Shape after taking 100 rows for each unique 'Question' (1000, 2)
      Question                                           Response
19008     ADHD  Hi! I would like to start by apologizing for m...
5834      ADHD                                          [removed]
2898      ADHD                                          [deleted]
7258      ADHD                                          [deleted]
29382     ADHD  Some back story. I have always been a "procras...
Shape of df3 (1000, 2)


In [10]:
#read the rules_facts.csv file and drop the null values and last column
df4 = pd.read_csv('../data/knowledge/reddit_types.csv')
print("Shape before dropping", df4.shape)
df4 = df4.dropna()
print("Shape after dropping", df4.shape)

#change the column names in to Question	and Response
df4 = df4.rename(columns={'comment':'Response'})
df4 = df4.rename(columns={'subreddit':'Question'})
print(df4.head())
# Take only the random 10 rows for each unique 'Question'
df4_grouped = df4.groupby('Question').sample(10)

print("Shape after taking 10 rows for each unique 'Question'", df4_grouped.shape)

#move 1st column as Question and 2nd column as Response
df4_grouped = df4_grouped[['Question', 'Response']]
print(df4_grouped.head())
print("Shape of df4",df4_grouped.shape)

Shape before dropping (7314363, 2)
Shape after dropping (7314354, 2)
  Question                                           Response
0   daddit     Your son needs to go explain this to congress.
1   daddit     "I drink your milkshake. I drink it right up."
2   daddit  Absolutely perfect analogy. /u/USER's son for ...
3   daddit                                          [deleted]
4   daddit  Except the milkshake store that sells the stra...
Shape after taking 10 rows for each unique 'Question' (1080, 2)
        Question                                           Response
5488739     ADHD  Man I’d love to work on a farm but I live in a...
5577902     ADHD  Stop putting expectations on yourself and you ...
5673876     ADHD  Being abusive is not a symptom of ADHD. This s...
5585006     ADHD  Just watched this with my kids tonight. I said...
5431175     ADHD  This particular thought I'm happy to easily lo...
Shape of df4 (1080, 2)


In [11]:
df5 = pd.read_csv("../data/sample_q_and_a/sentiment_analysis_1.csv")

# Lists to hold the question-answer pairs
questions = []
answers = []

# Iterate over the rows of the DataFrame
for i in range(len(df5) - 1):
    # Check if the current row's interlocutor is different from the next row's interlocutor
    if df5.iloc[i]['interlocutor'] != df5.iloc[i + 1]['interlocutor']:
        # Prepend the topic to the question text
        question = f"{df5.iloc[i]['topic']} {df5.iloc[i]['utterance_text']}"
        # Get the response text
        answer = df5.iloc[i + 1]['utterance_text']
        questions.append(question)
        answers.append(answer)

# Create a new DataFrame with the questions and answers
df5 = pd.DataFrame({
    'Question': questions,
    'Response': answers
})
print(df5.head())
print("Shape before dropping", df5.shape)
#drop nan values
df5 = df5.dropna()
print("Shape after dropping", df5.shape)

                                            Question  \
0  reducing recidivism Yeah, 'cause then we could...   
1  smoking cessation  -been helpful, um, in-- wit...   
2  reducing alcohol consumption Well, just I gues...   
3  reducing alcohol consumption Okay. Well, the r...   
4  unidentifiable I would say so, yeah. I don't k...   

                                            Response  
0  -been helpful, um, in-- with patients before. ...  
1  Well, just I guess, at a party or something, s...  
2  So, tell me a little bit more about your drink...  
3  I would say so, yeah. I don't know if I've eve...  
4  Not being a baby. This is a pretty normal- thi...  
Shape before dropping (4647, 2)
Shape after dropping (4647, 2)


In [12]:
#concatenate data and df in to single data frame
data=pd.read_csv('../data/sample_q_and_a/response_2.csv')
data = data.dropna()


In [13]:
#concatenate data and df in to single data frame
data = pd.concat([df1, data, df2, df3_grouped, df4_grouped, df5], ignore_index=True)
print(data.shape)

(24341, 2)


In [14]:
print(data.sample(5))
#drop text = NaN
print("Shape before dropping", data.shape)
data = data.dropna()
print("Shape after dropping", data.shape)


                                                Question  \
14055  \nOften times I find myself thinking scary tho...   
3190   I'm having trouble adjusting to a new life sit...   
3402   I'm not happy with my job and feel unfulfilled...   
6721    I'm feeling really overwhelmed and stressed out.   
1175   I'm having trouble with my social anxiety. I f...   

                                                Response  
14055  Hi there,I first want to let you know that hav...  
3190   Change can be tough, but we can work together ...  
3402   It's important to explore what's missing in yo...  
6721   It's understandable to feel overwhelmed in our...  
1175   Social anxiety can be difficult to manage, but...  
Shape before dropping (24341, 2)
Shape after dropping (24341, 2)


In [15]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24341 entries, 0 to 24340
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Question  24341 non-null  object
 1   Response  24341 non-null  object
dtypes: object(2)
memory usage: 380.5+ KB


Data preprocessing

In [16]:
# remove special characters, extra spaces, and make it lower case and remove the "deleted" and non-null rows
data = data[data['Question'] != '[deleted]']
data = data[data['Question'] != '[removed]']
data = data[data['Response'] != '[deleted]']
data = data[data['Response'] != '[removed]']
data = data[data['Question'] != '']
data = data[data['Response'] != '']
data = data[data['Question'] != 'nan']
data = data[data['Response'] != 'nan']
data = data[data['Question'] != 'NaN']
data = data[data['Response'] != 'NaN']
data = data.dropna()
data = data.drop_duplicates()
#remove special characters
data['Question'] = data['Question'].str.replace(r'\W', ' ') #remove special characters
data['Response'] = data['Response'].str.replace(r'\W', ' ') #remove special characters
data['Question'] = data['Question'].str.replace(r'\s+[a-zA-Z]\s+', ' ') #remove single characters
data['Response'] = data['Response'].str.replace(r'\s+[a-zA-Z]\s+', ' ') #remove single characters
#remove extra spaces
data['Question'] = data['Question'].str.replace(r'\s+', ' ') #remove extra spaces
data['Response'] = data['Response'].str.replace(r'\s+', ' ') #remove extra spaces
#remove leading and trailing spaces
data['Question'] = data['Question'].str.strip()
data['Response'] = data['Response'].str.strip()
#remove punctuations and digits and commas and inverted commas
data['Question'] = data['Question'].str.replace(r'[^a-zA-Z\s]', '') #remove punctuations and digits
data['Response'] = data['Response'].str.replace(r'[^a-zA-Z\s]', '') #remove punctuations and digits
#make it lower case

data['Question'] = data['Question'].str.lower()
data['Response'] = data['Response'].str.lower()
#remove rows with empty strings
data = data[data['Question'] != '']
data = data[data['Response'] != '']
data = data.drop_duplicates()


In [17]:
#print head and shape after preprocessing
print(data.head())
print("shape: \n", data.shape)

                                            Question  \
0  i'm feeling really anxious lately and i don't ...   
1  i think my partner may be cheating on me. what...   
2  i'm feeling really overwhelmed with work and s...   
3  i'm having trouble sleeping and i'm constantly...   
4  i've been feeling really anxious lately, and i...   

                                            Response  
0  it's common to feel anxious at times, and ther...  
1  it's understandable to feel worried and suspic...  
2  it sounds like you're going through a difficul...  
3  it's important to talk to your doctor about an...  
4  it's common to feel anxious without knowing th...  
shape: 
 (19953, 2)


In [18]:
data.to_csv('../data/sample_q_and_a/train_response.csv')


In [19]:
# Split the data into training and validation datasets
from sklearn.model_selection import train_test_split


train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

# Save the training and validation datasets to separate CSV files
train_data.to_csv('../data/sample_q_and_a/train_data.csv', index=False)
val_data.to_csv('../data/sample_q_and_a/val_data.csv', index=False)

In [20]:
train_file_path = '../data/sample_q_and_a/train_data.csv'
val_file_path = '../data/sample_q_and_a/val_data.csv'

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [17]:
def load_dataset(file_path, tokenizer, block_size = 1024):
    dataset_train = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset_train

In [18]:
def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, 
        mlm=mlm,
    )
    return data_collator

In [19]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Shift labels to the right and mask -100 values
    shift_logits = logits[..., :-1, :].contiguous()
    shift_labels = labels[..., 1:].contiguous()
    # Calculate Cross Entropy loss between predictions and labels
    loss_fct = torch.nn.CrossEntropyLoss(reduction='none')
    loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
    # Mask out the loss for -100 labels
    loss_mask = (shift_labels.view(-1) != -100)
    loss = loss[loss_mask].mean()
    # Calculate perplexity
    perplexity = torch.exp(loss)
    return {'perplexity': perplexity.item()}

In [20]:
def save_training_metrics(hist, output_dir):
    # Save the training metrics
    metrics_path = os.path.join(output_dir, 'training_metrics.json')
    with open(metrics_path, 'w') as f:
        json.dump(hist.metrics, f, indent=4)

In [21]:

def train(train_file_path, val_file_path, model_name, output_dir, overwrite_output_dir,
          per_device_train_batch_size, num_train_epochs):
    # Initialize tokenizer
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)

    # Initialize datasets
    train_dataset = load_dataset(train_file_path, tokenizer)
    val_dataset = load_dataset(val_file_path, tokenizer)

    # Initialize data collator
    data_collator = load_data_collator(tokenizer)

    # Initialize model
    model = GPT2LMHeadModel.from_pretrained(model_name).to(device)

    training_args = TrainingArguments(
        fp16=True,
        output_dir=output_dir,
        overwrite_output_dir=overwrite_output_dir,
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=per_device_train_batch_size,
        logging_dir="./logs",
        logging_steps=800,  # Log every 800 steps
        save_steps=800,  # Save checkpoint every 800 steps
        logging_first_step=True,
        save_total_limit=2,
        learning_rate=.0001
    )
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
    )

    # Train model
    hist = trainer.train()

    # Save model
    trainer.save_model()


    # Save model and tokenizer
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

    # Save training metrics
    save_training_metrics(hist, output_dir)

    # Print results
    print("Global Step:", hist.global_step)
    print("Train Loss:", hist.training_loss)

    return trainer, hist


In [22]:
model_name = "distilgpt2"  #GPT-2 model
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer

GPT2Tokenizer(name_or_path='distilgpt2', vocab_size=50257, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}

In [23]:
model = GPT2LMHeadModel.from_pretrained(model_name)
model
if torch.cuda.device_count() > 1:
    model = DataParallel(model)

model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [24]:
model.base_model


GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-5): 6 x GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
)

In [25]:
output_dir = '../data/sample_q_and_a/gpt-model'
overwrite_output_dir = True
per_device_train_batch_size = 2
num_train_epochs =  250    # Train
trainer, hist=train(
    train_file_path=train_file_path,
    val_file_path=val_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
) 

Step,Training Loss
1,3.656900
800,2.867700
1600,2.534300
2400,2.375800
3200,2.244800
4000,2.144200
4800,2.046700
5600,1.971400
6400,1.887100
7200,1.803300


In [1]:
print("Global Step:", hist.global_step)
print("Epoch:", hist.metrics['epoch'])
print("Train Runtime:", hist.metrics['train_runtime'])
print("Train Samples Per Second:", hist.metrics['train_samples_per_second'])
print("Train Steps Per Second:", hist.metrics['train_steps_per_second'])
print("Total FLOPS:", hist.metrics['total_flos'])
print("Train Loss:", hist.metrics['train_loss'])


NameError: name 'hist' is not defined

In [ ]:
# Evaluate the model and get metrics
metrics = trainer.evaluate().to(device)

# Print the evaluation loss
print('Evaluation loss:', metrics['eval_loss'])


OutOfMemoryError: CUDA out of memory. Tried to allocate 9.20 GiB. GPU 0 has a total capacty of 21.99 GiB of which 7.65 GiB is free. Including non-PyTorch memory, this process has 14.33 GiB memory in use. Of the allocated memory 10.45 GiB is allocated by PyTorch, and 3.57 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [2]:
# Plot train and eval loss graph
plt.plot(hist.metrics['train_loss'])
plt.plot(eval_metrics['eval_loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train Loss', 'Eval Loss'])
plt.show()

NameError: name 'plt' is not defined

In [21]:
def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model


def load_tokenizer(tokenizer_path):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    return tokenizer

def trim_incomplete_sentence(text):
    """
    Trims incomplete sentence from the end of the generated text.
    """
    # Attempt to find the last complete sentence
    for punct in [",",".","?","!", ";", ": ", ".\n", "?\n", "!\n", ";\n", ":\n", "\n", "\n\n", "\n\n\n", ", ", ". ", "? ", "! ", "; ", ": "]:
        last_punct_pos = text.rfind(punct)
        if last_punct_pos != -1:
            return text[:last_punct_pos + 5]
    return text  # Return the original text if no sentence-ending punctuation is found

def trim_extra_text(generated_text, max_sentences=2):
    sentences = sent_tokenize(generated_text)
    trimmed_text = ' '.join(sentences[:max_sentences])
    #remove sentence after inverted commas
    if '"' in trimmed_text:
        trimmed_text = trimmed_text[:trimmed_text.rfind('"')+1]
    return trimmed_text

def capitalize_sentences(text):
    sentences = sent_tokenize(text)
    capitalized_sentences = [sentence.capitalize() for sentence in sentences]
    return ' '.join(capitalized_sentences)

def generate_text(model_path, sequence, max_length, temperature=0.9):
    
    model = load_model(model_path)
    tokenizer = load_tokenizer(model_path)
    ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
    final_outputs = model.generate(
        ids,
        do_sample=True,
        temperature = temperature,
        max_length=max_length,
        pad_token_id=model.config.eos_token_id,
        eos_token_id=model.config.eos_token_id,
        top_k=100,
        top_p=0.6,
        early_stopping=True,  # Stops generation when EOS token is produced
        no_repeat_ngram_size=2
    )
    # Decoding the generated text
    generated_text = tokenizer.decode(final_outputs[0], skip_special_tokens=True)
    
    #removing question part from generated text
    clean_generated_text = generated_text[len(tokenizer.decode(ids[0], skip_special_tokens=True)):].strip()
    
    # Post-generation cleanup
    clean_generated_text = clean_generated_text.replace('","', '')
    clean_generated_text = clean_generated_text.replace(',', '')# Removing specific unwanted characters
    clean_generated_text = trim_incomplete_sentence(clean_generated_text)  # Trimming incomplete sentences
    clean_generated_text = trim_extra_text(clean_generated_text, max_sentences=3)  # Trimming extra text
    
    #captialize the words at start of sentence
    clean_generated_text = capitalize_sentences(clean_generated_text)    
    
    return clean_generated_text

In [22]:
model_path = "../data/sample_q_and_a/gpt-model"
sequence = data['Question'].iloc[10]
max_len = 100
print("Q : ",data['Question'].iloc[10])
print()
print("A : ",data['Response'].iloc[10])
print()
print("G : ",generate_text(model_path, sequence, max_len))

Q :  i feel like i'm not good enough at my job and it's causing me a lot of stress. what can i do to feel more confident?

A :  it's common to feel insecure about our abilities from time to time. let's work together to identify your strengths and accomplishments, and develop a plan for building your confidence. we can also explore any underlying beliefs or thought patterns that may be contributing to your self-doubt.

G :  "it's common to experience self-doubt at work but it important to remember that these thoughts are not always accurate. Try to focus on your strengths and accomplishments and practice self selfcompassion. You may also benefit from talking to a therapist who can help you work through these feelings and develop strategies for building your confidence."


In [24]:
sequence = input("Enter the question : ")
max_len = 500
print()
print("G : ",generate_text(model_path, sequence, max_len))

STREAMLIT-APP

In [ ]:
# Define keywords for detecting crisis situations
CRISIS_KEYWORDS = ['suicide', 'selfharm', 'self-harm', 'assault', 'suicidal']

# Define a function to check for crisis keywords and return appropriate responses
def check_for_crisis(text):
    if any(keyword in text.lower() for keyword in CRISIS_KEYWORDS):
        return ("If you're in crisis or think you may have an emergency, call your doctor or 911 immediately. "
                "If you're having suicidal thoughts, call 1-800-273-TALK (1-800-273-8255) in the U.S. "
                "to talk to a skilled, trained counselor at a crisis center in your area at any time (National Suicide Prevention Lifeline). "
                "If you are located outside the United States, call your local emergency line immediately.")
    return None

st.title('Mental Health Support Chat')

In [ ]:
# Greeting message at the start of the chat
if 'start_message' not in st.session_state:
    st.session_state['start_message'] = "Hi! How are you feeling today?"

# Goodbye message at the end of the chat
if 'end_message' not in st.session_state:
    st.session_state['end_message'] = "It was nice talking to you. Have a good day!"

# Display the start message once at the beginning
if 'start_message' in st.session_state and st.session_state['start_message']:
    st.write(st.session_state['start_message'])
    st.session_state['start_message'] = None  # Clear the message so it doesn't show again

# Text input for user's message
user_input = st.text_input("Enter your message or type 'bye' to end the chat:")

# Main chat function
if st.button('Send'):
    # End the chat if the user says goodbye
    if user_input.lower() in ["bye", "goodbye", "quit"]:
        st.write(st.session_state['end_message'])
        st.stop()

    # Check for crisis situations and respond accordingly
    crisis_response = check_for_crisis(user_input)
    if crisis_response:
        st.write(crisis_response)
        st.stop()

    # Otherwise, generate a response using the model
    else:
        # Set the model path (adjust as necessary)
        model_path = "../data/sample_q_and_a/gpt-model"
        generated_response = generate_text(model_path, user_input, max_len=100)
        st.text_area("AI Response:", value=generated_response, height=150)
